<a href="https://colab.research.google.com/github/asifshigri12/NLP/blob/main/case_study_for_NLP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('/content/SPAM text message 20170820 - Data.csv')
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [3]:
import string

In [4]:
#Function to normalize text
def normalize_text(text):
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    return text

In [7]:
#Apply normalization
df['text_normalized'] = df['Message'].apply(normalize_text)


Tokenize and Remove Stopwords

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [9]:
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
stop_words = set(stopwords.words('english'))


In [11]:
# Function to tokenize and remove stopwords
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [12]:
# Apply tokenization and stopword removal
df['text_tokenized'] = df['text_normalized'].apply(tokenize_and_remove_stopwords)

Stemming and Lemmatization

In [13]:
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('wordnet')

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function to apply stemming and lemmatization
def stem_and_lemmatize(tokens):
    stemmed = [ps.stem(token) for token in tokens]
    lemmatized = [lemmatizer.lemmatize(token) for token in stemmed]
    return lemmatized

# Apply stemming and lemmatization
df['text_processed'] = df['text_tokenized'].apply(stem_and_lemmatize)


[nltk_data] Downloading package wordnet to /root/nltk_data...


Transform Text Data using N-gram Models

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

# Convert processed tokens back to string
df['text_processed_str'] = df['text_processed'].apply(lambda x: ' '.join(x))

# Create N-gram models
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
trigram_vectorizer = CountVectorizer(ngram_range=(1, 3))

# Transform text data
X_unigram = unigram_vectorizer.fit_transform(df['text_processed_str'])
X_bigram = bigram_vectorizer.fit_transform(df['text_processed_str'])
X_trigram = trigram_vectorizer.fit_transform(df['text_processed_str'])


Split the Data into Training and Testing Sets

In [16]:
from sklearn.model_selection import train_test_split

# Assuming 'label' column has the spam/ham labels
X_train, X_test, y_train, y_test = train_test_split(X_unigram, df['Message'], test_size=0.2, random_state=42)


In [17]:
from sklearn.linear_model import LogisticRegression

# Train the classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)


LogisticRegression()

 Evaluate the Model

In [18]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the test set
y_pred = clf.predict(X_test)


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy: 0.09327354260089686
Precision: 0.07560523725009072
Recall: 0.09327354260089686
F1-Score: 0.07876521398176767


Preprocessed Dataset:

In [21]:
df.to_csv('preprocessed_dataset.csv', index=False)


In [22]:
print(df['Message'].unique())


['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'
 'Ok lar... Joking wif u oni...'
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
 ... 'Pity, * was in mood for that. So...any other suggestions?'
 "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"
 'Rofl. Its true to its name']
